In [1]:
import numpy as np
import pandas as pd
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 10)

# Ensembl 98 mouse Averaged dN/dS

In [3]:
# dN/dS of all protein coding genes
dnds_df = pd.read_excel('/Users/xulinhe/OneDrive/Herculano_Lab/Ensembl98.mouse.xlsx',usecols=['Gene name','avg_dNdS'],sep='\t',index_col=0)

In [4]:
dnds_df

,avg_dNdS
Gene name,
mt-Cytb,0.020620
mt-Nd6,0.016243
mt-Nd5,0.040241
mt-Nd4l,0.013863
mt-Nd3,0.037464
...,...
Kmt2b,0.131660
Fhl4,0.510940
Phc3,0.092761


## Bechmarks 

### House keeping genes

In [8]:
# house-keeping genes
hk_df = pd.read_csv('/Users/xulinhe/OneDrive/Herculano_Lab/Benchmark_Genes/Eisenberg_et_Levanon_2013/trimmed.mouse_ortholog_house_keeping_genes.tsv',sep='\t',usecols=['Gene name'],index_col=['Gene name'])

In [10]:
hk_df = pd.merge(hk_df,dnds_df,left_index=True, right_index=True, how='inner')

In [11]:
hk_df

,avg_dNdS
Gene name,
Aaas,0.088135
Aagab,0.135716
Aamp,0.022372
Aar2,0.093255
Aars,0.047271
...,...
Zswim1,0.112086
Zswim7,0.211968
Zswim8,0.060767


### MHC genes

In [12]:
# mouse MHC Genes
mhc_df = pd.read_csv('/Users/xulinhe/OneDrive/Herculano_Lab/Benchmark_Genes/MHC/Shiina_et_al_2017/ensembl_ID.mouse.MHC_genes.tsv',sep='\t', usecols=['Gene name'],index_col=['Gene name'])

In [13]:
mhc_df = pd.merge(mhc_df,dnds_df,left_index=True, right_index=True, how='inner')

In [14]:
mhc_df

,avg_dNdS
Gene name,
H2-Aa,0.448644
H2-Ab1,0.254874
H2-DMa,0.234635
H2-DMb1,0.701711
H2-DMb2,0.519831
...,...
H2-Q1,0.304375
H2-Q2,0.467251
H2-T23,0.488723


### ATPase

In [15]:
# HGNC database, accessed Oct. 10, 2019
atpase_df = pd.read_csv('/Users/xulinhe/OneDrive/Herculano_Lab/Benchmark_Genes/ATPase/HGNC_database/Ensembl_ID.mouse.ATPase.tsv',sep='\t',usecols=['Gene name'],index_col=['Gene name'])

In [16]:
atpase_df = pd.merge(atpase_df,dnds_df,left_index=True,right_index=True,how='inner')

In [17]:
atpase_df

,avg_dNdS
Gene name,
Afg3l2,0.054694
Atad1,0.017051
Atad2b,0.105365
Atad3a,0.050244
Atad5,0.317224
...,...
Vps4b,0.056053
Wrnip1,0.055883
Yme1l1,0.070402


### Genes in GO immune system from MGI database

In [20]:
# innate immune genes from https://www.innatedb.com/annotatedGenes.do?type=innatedb, accessed Jan 22, 2020
immune_df = pd.read_csv('/Users/xulinhe/OneDrive/Herculano_Lab/Benchmark_Genes/InnateDB_innate_immune_genes.txt',usecols=['species','name'],sep='\t')

In [21]:
immune_df

,species,name
0,Mus musculus,Map3k8
1,Mus musculus,Rictor
2,Mus musculus,Lyn
3,Mus musculus,Unc93b1
4,Mus musculus,Gata6
...,...,...
2276,Bos taurus,BT.87655
2277,Bos taurus,LOC618985
2278,Bos taurus,IFNA16
2279,Bos taurus,BT.63938


In [24]:
# trimming out all non-mouse genes
immune_df = immune_df.loc[immune_df['species']=='Mus musculus'].copy(deep=True)

In [25]:
immune_df.rename(columns = {'name':'Gene name'}, inplace = True)

In [26]:
immune_df = pd.merge(immune_df,dnds_df,left_on=['Gene name'],right_index=True,how='inner')

In [27]:
immune_df.set_index('Gene name', inplace=True)

In [30]:
immune_df.drop(columns=['species'],inplace=True)

In [31]:
immune_df

,avg_dNdS
Gene name,
Map3k8,0.059851
Rictor,0.046778
Lyn,0.034117
Unc93b1,0.073698
Gata6,0.095946
...,...
Ifna2,0.433356
Ifna14,0.421975
Ifna9,0.382669


# Export Dataframes of Each Benchmark to a Sheet of Excel File

In [33]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('/Users/xulinhe/OneDrive/Herculano_Lab/benchmarks.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
hk_df.to_excel(writer, sheet_name='House Keeping Genes')
mhc_df.to_excel(writer, sheet_name='MHC')
atpase_df.to_excel(writer, sheet_name='ATPase')
immune_df.to_excel(writer, sheet_name='Immune System Genes')

# Close the Pandas Excel writer and output the Excel file.
writer.save()